In [1]:
import psycopg2
from decouple import config
import pandas as pd

In [2]:
with psycopg2.connect(
		dbname=config("DB_DB"),
		user=config("DB_USER"),
		password=config("DB_PASSWORD"),
		host=config("DB_HOST"),
		port=config("DB_PORT")
) as psql_conn:
    
    curs = psql_conn.cursor()
    curs.execute("""
        SELECT COUNT(*)
        FROM job_listings
    """)
    print(curs.fetchone())

(118888,)


In [3]:
conn = psycopg2.connect(
		dbname=config("DB_DB"),
		user=config("DB_USER"),
		password=config("DB_PASSWORD"),
		host=config("DB_HOST"),
		port=config("DB_PORT")
)
df = pd.read_sql("""
    SELECT *
    FROM job_listings;
""", conn)



In [4]:
with open("job_listings.csv", "w") as f:
    df.to_csv(f)

In [7]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)

In [9]:
import re

def tokenize(doc):
    return [
        re.sub(r"[^a-z0-9]", "", t.lemma_.lower()).strip() for t in tokenizer(doc)
        if not t.is_stop and not t.is_punct and t.text.strip()
    ]

df["tokens"] = df["title"].apply(tokenize)
df.head()

,id,post_date_utc,pay_min,pay_max,pay_exact,title,seniority,tokens
0,1,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,"[software, dev]"
1,2,2020-01-31 21:55:07.257399+00:00,60000.0,190000.0,120000.0,node developer,senior,"[node, developer]"
2,3,2020-01-31 21:55:07.257399+00:00,40000.0,90000.0,80000.0,ux designer,some guy,"[ux, designer]"
3,4,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,"[software, dev]"
4,5,2020-01-31 21:55:07.257399+00:00,60000.0,190000.0,120000.0,node developer,senior,"[node, developer]"


In [10]:
df["is_nurse"] = df["tokens"].apply(lambda x: "nurse" in x)

In [11]:
df["is_nurse"].sum()

1077

In [13]:
nurse_rows = df[df["is_nurse"]].index

In [19]:
conn = psycopg2.connect(
		dbname=config("DB_DB"),
		user=config("DB_USER"),
		password=config("DB_PASSWORD"),
		host=config("DB_HOST"),
		port=config("DB_PORT")
)
curs = conn.cursor()

curs.execute("""
    SELECT *
    FROM lda17_topics
""")

In [22]:
lda_df = pd.read_sql("""
    SELECT * FROM lda17_topics
""", conn)
lda_df.head()

,id,job_id,lda0,lda1,lda2,lda3,lda4,lda5,lda6,lda7,lda8,lda9,lda10,lda11,lda12,lda13,lda14,lda15,lda16,in_nn
0,95311,96895,0.001943,0.001623,0.001395,0.983966,0.001089,0.000982,0.000893,0.000820,0.000757,0.000704,0.000657,0.000616,0.000580,0.000548,0.000520,0.000494,0.002413,True
1,95312,96896,0.265231,0.000857,0.100442,0.106023,0.215209,0.000519,0.000472,0.000433,0.000400,0.000372,0.307309,0.000326,0.000307,0.000290,0.000274,0.000261,0.001276,True
2,95313,96897,0.000356,0.000297,0.000256,0.086600,0.104553,0.431239,0.000164,0.000150,0.000139,0.000129,0.281771,0.000113,0.000106,0.000100,0.000095,0.000090,0.093843,True
3,95314,96898,0.082803,0.000488,0.000419,0.193798,0.213526,0.000295,0.000269,0.000246,0.140362,0.000212,0.034349,0.044669,0.000174,0.000165,0.287350,0.000148,0.000727,True
4,95315,96899,0.028781,0.000341,0.000293,0.000257,0.490687,0.000207,0.010322,0.000172,0.000159,0.000148,0.427289,0.040383,0.000122,0.000115,0.000109,0.000104,0.000508,True


In [23]:
curs = conn.cursor()
execute_batch(curs, """
    DELETE FROM lda17_topics
    WHERE job_id=%s;
""", [[ix] for ix in nurse_rows])
execute_batch(curs, """
    DELETE FROM job_listings
    WHERE id=%s;
""", [[ix] for ix in nurse_rows])

curs.close()
conn.commit()

In [26]:
curs = conn.cursor()

curs.execute("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema='public';
""")
curs.fetchall()

[('knex_migrations',),
 ('knex_migrations_lock',),
 ('companies',),
 ('users',),
 ('locations',),
 ('job_companies',),
 ('job_listings',),
 ('job_descriptions',),
 ('job_keyphrases',),
 ('user_jobs',),
 ('job_locations',),
 ('user_skills',),
 ('skills',),
 ('job_links',),
 ('lda17_topics',)]

In [58]:

df = pd.read_sql("""
    SELECT *
    FROM job_listings;
""", conn)

descriptions_df = pd.read_sql("""
    SELECT *
    FROM job_descriptions;
""", conn)
companies_df = pd.read_sql("""
    SELECT *
    FROM job_companies;
""", conn)
keyphrases_df = pd.read_sql("""
    SELECT *
    FROM job_keyphrases;
""", conn)
locations_df = pd.read_sql("""
    SELECT *
    FROM job_locations;
""", conn)
descriptions_df.head()

,id,job_id,description
0,1,1,job 1 this is a great place to work
1,2,2,job 2 this is a great place to work
2,3,3,job 3 this is a great place to work
3,4,4,job 4 this is a great place to work
4,5,5,job 5 this is a great place to work


In [59]:
companies_df.head()

,id,company_id,job_id
0,1,1,1
1,2,2,2
2,3,3,3
3,4,1,4
4,5,2,5


In [60]:
keyphrases_df.head()

,id,job_id,keyphrase
0,1,1,"java, ruby"
1,2,2,"java, ruby"
2,3,3,"java, ruby"


In [61]:
locations_df.head()

,id,job_id,location_id
0,1,1,1
1,2,2,2
2,3,3,3
3,4,4,1
4,5,5,2


In [62]:
df = df.merge(descriptions_df, left_on="id", right_on="job_id", how="outer")

In [63]:
df = df.merge(companies_df, left_on="job_id", right_on="job_id", how="outer")
df = df.merge(keyphrases_df, left_on="job_id", right_on="job_id", how="outer")
df = df.merge(locations_df, left_on="job_id", right_on="job_id", how="outer")

In [64]:
for col in df.columns:
    print(f"\"{col}\"")

"id_x"
"post_date_utc"
"pay_min"
"pay_max"
"pay_exact"
"title"
"seniority"
"id_y"
"job_id"
"description"
"id_x"
"company_id"
"id_y"
"keyphrase"
"id"
"location_id"


In [65]:
df = df.drop(columns=["id", "id_x", "id_y"])
df.head()

,post_date_utc,pay_min,pay_max,pay_exact,title,seniority,job_id,description,company_id,keyphrase,location_id
0,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,1,job 1 this is a great place to work,1,"java, ruby",1
1,2020-01-31 21:55:07.257399+00:00,60000.0,190000.0,120000.0,node developer,senior,2,job 2 this is a great place to work,2,"java, ruby",2
2,2020-01-31 21:55:07.257399+00:00,40000.0,90000.0,80000.0,ux designer,some guy,3,job 3 this is a great place to work,3,"java, ruby",3
3,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,4,job 4 this is a great place to work,1,NaN,1
4,2020-01-31 21:55:07.257399+00:00,60000.0,190000.0,120000.0,node developer,senior,5,job 5 this is a great place to work,2,NaN,2


In [66]:
companies_df2 = pd.read_sql("""
    SELECT *
    FROM companies;
""", conn)

In [67]:
df = df.merge(companies_df2, left_on="company_id", right_on="id", how="outer")

In [68]:
df.head()

,post_date_utc,pay_min,pay_max,pay_exact,title,seniority,job_id,description_x,company_id,keyphrase,location_id,id,name,description_y,size,revenue,logo_url
0,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,1.0,job 1 this is a great place to work,1.0,"java, ruby",1.0,1,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...
1,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,4.0,job 4 this is a great place to work,1.0,NaN,1.0,1,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...
2,2020-02-04 18:27:17+00:00,NaN,NaN,NaN,Research Software Engineer,None,2034.0,**Introduction**\n \nIBM Research Scientists ...,1.0,NaN,835.0,1,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...
3,2020-02-04 20:00:45+00:00,NaN,NaN,NaN,2020 Intern - Back End Developer,None,2473.0,**Introduction**\n \nSoftware Developers at I...,1.0,NaN,1045.0,1,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...
4,2020-02-04 20:01:52+00:00,NaN,NaN,NaN,Entry Level - Back End Developer,None,2493.0,**Introduction**\n \nSoftware Developers at I...,1.0,NaN,688.0,1,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...


In [69]:
locations_df2 = pd.read_sql("""
    SELECT *
    FROM locations;
""", conn)
df = df.merge(locations_df2, left_on="company_id", right_on="id", how="outer")
df.head()

,post_date_utc,pay_min,pay_max,pay_exact,title,seniority,job_id,description_x,company_id,keyphrase,...,id_x,name,description_y,size,revenue,logo_url,id_y,city,state_province,country
0,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,1.0,job 1 this is a great place to work,1.0,"java, ruby",...,1.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,1.0,oakland,california,USA
1,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,4.0,job 4 this is a great place to work,1.0,NaN,...,1.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,1.0,oakland,california,USA
2,2020-02-04 18:27:17+00:00,NaN,NaN,NaN,Research Software Engineer,None,2034.0,**Introduction**\n \nIBM Research Scientists ...,1.0,NaN,...,1.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,1.0,oakland,california,USA
3,2020-02-04 20:00:45+00:00,NaN,NaN,NaN,2020 Intern - Back End Developer,None,2473.0,**Introduction**\n \nSoftware Developers at I...,1.0,NaN,...,1.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,1.0,oakland,california,USA
4,2020-02-04 20:01:52+00:00,NaN,NaN,NaN,Entry Level - Back End Developer,None,2493.0,**Introduction**\n \nSoftware Developers at I...,1.0,NaN,...,1.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,1.0,oakland,california,USA


In [70]:
df = df.drop(columns=["id_x", "id_y"])


In [79]:
df = df.rename(
    columns={
        "description_x": "job_description",
        "description_y": "company_description",
    }
)

In [80]:
df.head()

,post_date_utc,pay_min,pay_max,pay_exact,title,seniority,job_id,job_description,company_id,keyphrase,location_id,name,company_description,size,revenue,logo_url,city,state_province,country
0,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,1.0,job 1 this is a great place to work,1.0,"java, ruby",1.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,oakland,california,USA
1,2020-01-31 21:55:07.257399+00:00,30000.0,100000.0,89000.0,software dev,junior,4.0,job 4 this is a great place to work,1.0,NaN,1.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,oakland,california,USA
2,2020-02-04 18:27:17+00:00,NaN,NaN,NaN,Research Software Engineer,None,2034.0,**Introduction**\n \nIBM Research Scientists ...,1.0,NaN,835.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,oakland,california,USA
3,2020-02-04 20:00:45+00:00,NaN,NaN,NaN,2020 Intern - Back End Developer,None,2473.0,**Introduction**\n \nSoftware Developers at I...,1.0,NaN,1045.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,oakland,california,USA
4,2020-02-04 20:01:52+00:00,NaN,NaN,NaN,Entry Level - Back End Developer,None,2493.0,**Introduction**\n \nSoftware Developers at I...,1.0,NaN,688.0,IBM,IBM is the largest technology and consulting e...,500000.0,7.900000e+10,https://coda.newjobs.com/api/imagesproxy/ms/mm...,oakland,california,USA


In [73]:
df.shape

(118782, 19)

In [74]:
with open("job_listings.csv", "w") as f:
    df.to_csv(f)

In [81]:
df["job_description"].str.replace(r"\s+", " ")

0                       job 1 this is a great place to work
1                       job 4 this is a great place to work
2         **Introduction**    IBM Research Scientists ar...
3         **Introduction**    Software Developers at IBM...
4         **Introduction**    Software Developers at IBM...
                                ...                        
118777                                                  NaN
118778                                                  NaN
118779                                                  NaN
118780                                                  NaN
118781                                                  NaN
Name: job_description, Length: 118782, dtype: object